# Vector Stores

Hướng dẫn này cung cấp một hướng dẫn toàn diện về **vector stores**, là các cơ sở dữ liệu chuyên dụng để lập chỉ mục và truy xuất thông tin bằng cách sử dụng biểu diễn vector (**embeddings**). Hướng dẫn này nhấn mạnh tầm quan trọng của chúng trong việc cho phép tìm kiếm ngữ nghĩa nhanh chóng, có khả năng mở rộng và theo ngữ cảnh trên dữ liệu phi cấu trúc như văn bản, hình ảnh và âm thanh.

### Các phần chính:

1.  **Essentials of Vector Stores (Các yếu tố cần thiết của Vector Stores)**:
    -   Tập trung vào nhu cầu tìm kiếm ngữ nghĩa (**semantic search**), khả năng mở rộng (**scalability**) và truy xuất hiệu quả (**efficient retrieval**).
    -   Tổng quan về cách **vector stores** vượt trội hơn các hệ thống tìm kiếm dựa trên từ khóa truyền thống (**traditional keyword-based search systems**).

2.  **LangChain Interface (Giao diện LangChain)**:
    -   Giải thích các phương thức cốt lõi như `add_documents`, `upsert_documents`, `delete_documents` và `similarity_search`.
    -   Giới thiệu các tính năng nâng cao như xử lý tài liệu song song (**parallel document processing**).

3.  **Search Methods (Phương pháp tìm kiếm)**:
    -   Bao gồm `Keyword-Based Search` (Tìm kiếm dựa trên từ khóa), dựa trên việc khớp chính xác các thuật ngữ, và `Similarity-Based Search` (Tìm kiếm dựa trên độ tương đồng), xác định các kết quả liên quan về mặt ngữ nghĩa bằng cách sử dụng **embeddings**.
    -   Giải thích `Score-Based Similarity Search` (Tìm kiếm độ tương đồng dựa trên điểm số), xếp hạng tài liệu dựa trên mức độ liên quan bằng cách sử dụng các chỉ số như `cosine similarity` (độ tương đồng cosine).
    -   Thảo luận về các kỹ thuật tìm kiếm nâng cao, bao gồm `Sparse` (Thưa thớt), `Dense` (Dày đặc) và `Hybrid Search` (Tìm kiếm kết hợp), để cải thiện hiệu suất truy xuất.
    -   Minh họa việc sử dụng phương thức `.as_retriever()` của LangChain để tích hợp liền mạch với các pipeline truy xuất.

5.  **Integration of Popular Vector Stores (Tích hợp các Vector Stores phổ biến)**:
    -   Tổng quan về các **vector stores** nổi bật như Chroma, FAISS, Pinecone, Qdrant, Elasticsearch, MongoDB, pgvector, Neo4j, Weaviate và Milvus.
    -   Điểm mạnh và các trường hợp sử dụng phù hợp của mỗi kho được tóm tắt ngắn gọn.

Hướng dẫn này đóng vai trò là hướng dẫn cơ bản để hiểu và tận dụng **vector stores** cho các ứng dụng dựa trên AI. Nó kết nối các khái niệm cơ bản với các triển khai nâng cao, cung cấp thông tin chi tiết về các chiến lược truy xuất và tích hợp dữ liệu hiệu quả trong hệ sinh thái của LangChain.


## Hướng dẫn khái niệm về Vector Store

*Điều kiện tiên quyết: `07-TextSplitter` `08-Embedding`*

**Vector stores** là các cơ sở dữ liệu chuyên dụng được thiết kế để **index** (lập chỉ mục) và **retrieve** (truy xuất) thông tin bằng cách sử dụng biểu diễn vector (**embeddings**).

Chúng thường được sử dụng để tìm kiếm dữ liệu phi cấu trúc, chẳng hạn như văn bản, hình ảnh và âm thanh, bằng cách xác định nội dung tương tự về mặt ngữ nghĩa thay vì dựa vào việc khớp chính xác từ khóa.


### Tại sao Vector Store lại cần thiết

1.  Tìm kiếm nhanh chóng và hiệu quả

    Bằng cách lưu trữ và lập chỉ mục các vector embedding một cách phù hợp, **vector stores** cho phép truy xuất nhanh chóng thông tin liên quan, ngay cả khi xử lý các tập dữ liệu khổng lồ.

2.  Khả năng mở rộng cho dữ liệu ngày càng tăng

    Khi dữ liệu tiếp tục mở rộng, **vector stores** phải mở rộng quy mô một cách hiệu quả. Một **vector store** có cấu trúc tốt đảm bảo hệ thống có thể xử lý dữ liệu quy mô lớn mà không gặp vấn đề về hiệu suất, hỗ trợ tăng trưởng liền mạch.

3.  Tạo điều kiện cho Tìm kiếm Ngữ nghĩa (**Semantic Search**)

    Không giống như tìm kiếm dựa trên từ khóa truyền thống, tìm kiếm ngữ nghĩa truy xuất nội dung dựa trên ý nghĩa. **Vector stores** cho phép điều này bằng cách tìm các đoạn văn hoặc phần phù hợp chặt chẽ với truy vấn của người dùng trong ngữ cảnh. Đây là một lợi thế quan trọng so với các cơ sở dữ liệu lưu trữ văn bản thô, vốn bị giới hạn ở việc khớp chính xác từ khóa.


### Interface

LangChain cung cấp một giao diện thống nhất để tương tác với **vector stores**, cho phép người dùng chuyển đổi liền mạch giữa các triển khai khác nhau.

Giao diện này bao gồm các phương thức cốt lõi để **writing** (ghi), **deleting** (xóa) và **searching** (tìm kiếm) tài liệu trong **vector store**.

Các phương thức chính như sau:

-   `add_documents`: Thêm danh sách văn bản vào **vector store**.
-   `upsert_documents`: Thêm các tài liệu mới vào **vector store** hoặc cập nhật các tài liệu hiện có nếu chúng đã tồn tại.
    -   Trong hướng dẫn này, chúng ta cũng sẽ giới thiệu phương thức `upsert_documents_parallel`, cho phép xử lý hàng loạt dữ liệu hiệu quả khi áp dụng.
-   `delete_documents`: Xóa danh sách tài liệu khỏi **vector store**.
-   `similarity_search`: Tìm kiếm các tài liệu tương tự với một truy vấn nhất định.

### Hiểu các phương pháp tìm kiếm

-   **Keyword-Based Search (Tìm kiếm dựa trên từ khóa)**
    Phương pháp này dựa trên việc khớp các từ hoặc cụm từ chính xác trong truy vấn với những từ hoặc cụm từ trong tài liệu. Nó đơn giản nhưng không có khả năng nắm bắt các mối quan hệ ngữ nghĩa giữa các thuật ngữ.

-   **Similarity-Based Search (Tìm kiếm dựa trên độ tương đồng)**
    Sử dụng biểu diễn vector để đánh giá mức độ tương tự về mặt ngữ nghĩa của truy vấn với các tài liệu. Nó cung cấp kết quả chính xác hơn, đặc biệt đối với các truy vấn ngôn ngữ tự nhiên.

-   **Score-Based Similarity Search (Tìm kiếm độ tương đồng dựa trên điểm số)**
    Gán điểm tương đồng cho mỗi tài liệu dựa trên truy vấn. Điểm số cao hơn cho thấy mức độ liên quan mạnh hơn. Thường sử dụng các chỉ số như `cosine similarity` (độ tương đồng cosine) hoặc `distance-based scoring` (chấm điểm dựa trên khoảng cách).

### Cách thức hoạt động của tìm kiếm độ tương đồng

-   **Khái niệm về Embeddings và Vectors (Nhúng và Vectors)**
    `Embeddings` là biểu diễn số của các từ hoặc tài liệu trong không gian đa chiều. Chúng nắm bắt ý nghĩa ngữ nghĩa, cho phép so sánh tốt hơn giữa truy vấn và tài liệu.

-   **Phương pháp đo độ tương đồng**
    -   **Cosine Similarity (Độ tương đồng cosine)**: Đo cosine của góc giữa hai vector. Các giá trị gần 1 hơn cho thấy độ tương đồng cao hơn.
    -   **Euclidean Distance (Khoảng cách Euclidean)**: Tính khoảng cách đường thẳng giữa hai điểm trong không gian vector. Khoảng cách nhỏ hơn ngụ ý độ tương đồng cao hơn.

-   **Chấm điểm và xếp hạng kết quả tìm kiếm**
    Sau khi tính toán độ tương đồng, các tài liệu được gán điểm số. Kết quả được xếp hạng theo thứ tự giảm dần mức độ liên quan dựa trên các điểm số này.

-   **Tổng quan ngắn gọn về các thuật toán tìm kiếm**
    -   `TF-IDF`: Trọng số các thuật ngữ dựa trên tần suất của chúng trong một tài liệu so với sự xuất hiện của chúng trên tất cả các tài liệu.
    -   `BM25`: Phiên bản tinh chỉnh của `TF-IDF`, được tối ưu hóa cho mức độ liên quan trong truy xuất thông tin.
    -   `Neural Search (Tìm kiếm thần kinh)`: Tận dụng học sâu để tạo `embeddings` nhận biết ngữ cảnh cho kết quả chính xác hơn.

### Các loại tìm kiếm trong Vector Stores

-   **Similarity Search (Tìm kiếm độ tương đồng)**: Tìm các tài liệu có `embeddings` giống nhất với truy vấn. Lý tưởng cho các ứng dụng tìm kiếm ngữ nghĩa.

-   **Maximal Marginal Relevance (MMR) Search (Tìm kiếm mức độ liên quan cận biên tối đa)**: Cân bằng mức độ liên quan và tính đa dạng trong kết quả tìm kiếm bằng cách ưu tiên các tài liệu đa dạng nhưng có liên quan.

-   **Sparse Retriever (Trình truy xuất thưa thớt)**: Sử dụng các phương pháp dựa trên từ khóa truyền thống như `TF-IDF` hoặc `BM25` để truy xuất tài liệu. Hiệu quả đối với các tập dữ liệu có ngữ cảnh hạn chế.

-   **Dense Retriever (Trình truy xuất dày đặc)**: Dựa vào `embeddings` vector dày đặc để nắm bắt ý nghĩa ngữ nghĩa. Phổ biến trong các hệ thống tìm kiếm hiện đại sử dụng học sâu.

-   **Hybrid Search (Tìm kiếm kết hợp)**: Kết hợp các phương pháp truy xuất thưa thớt và dày đặc. Cân bằng độ chính xác của các phương pháp dày đặc với phạm vi bao phủ rộng của các phương pháp thưa thớt để có kết quả tối ưu.

### Vector Store như một Retriever (Trình truy xuất)

-   **Chức năng**: Bằng cách chuyển đổi **vector store** thành trình truy xuất bằng phương thức `.as_retriever()`, bạn tạo một lớp bao bọc gọn nhẹ tuân thủ giao diện trình truy xuất của LangChain. Điều này cho phép sử dụng các chiến lược truy xuất khác nhau, chẳng hạn như `similarity search` và `maximal marginal relevance (MMR) search`, đồng thời cho phép tùy chỉnh các tham số truy xuất.
-   **Trường hợp sử dụng**: Lý tưởng cho các ứng dụng phức tạp trong đó trình truy xuất cần là một phần của pipeline lớn hơn, chẳng hạn như hệ thống tạo được tăng cường truy xuất (RAG). Nó tạo điều kiện tích hợp liền mạch với các thành phần khác trong LangChain, cho phép các chức năng như phương pháp truy xuất tập hợp và phân tích truy vấn nâng cao.

Tóm lại, trong khi các tìm kiếm **vector store** trực tiếp cung cấp khả năng truy xuất cơ bản, việc chuyển đổi **vector store** thành trình truy xuất mang lại tính linh hoạt và khả năng tích hợp nâng cao trong hệ sinh thái của LangChain, hỗ trợ các chiến lược và ứng dụng truy xuất phức tạp hơn.


## Integration

Dưới đây là tổng quan ngắn gọn về các **vector stores** được đề cập trong hướng dẫn này:

-   `Chroma`: Một cơ sở dữ liệu vector mã nguồn mở được thiết kế cho các ứng dụng AI, cho phép lưu trữ và truy xuất **embeddings** hiệu quả.
-   `FAISS`: Được phát triển bởi Facebook AI, FAISS (Facebook AI Similarity Search) là một thư viện để tìm kiếm độ tương đồng và phân cụm vector dày đặc hiệu quả.
-   `Pinecone`: Một dịch vụ cơ sở dữ liệu vector được quản lý cung cấp tìm kiếm độ tương đồng vector hiệu suất cao, cho phép các nhà phát triển xây dựng các ứng dụng AI có khả năng mở rộng.
-   `Qdrant`: Qdrant (đọc là: quadrant) là một công cụ tìm kiếm độ tương đồng vector. Nó cung cấp một dịch vụ sẵn sàng sản xuất với API tiện lợi để lưu trữ, tìm kiếm và quản lý vector với tải trọng bổ sung và hỗ trợ lọc mở rộng. Nó hữu ích cho tất cả các loại kết hợp dựa trên mạng thần kinh hoặc ngữ nghĩa, tìm kiếm theo khía cạnh và các ứng dụng khác.
-   `Elasticsearch`: Một công cụ tìm kiếm và phân tích phân tán, RESTful hỗ trợ tìm kiếm vector, cho phép tìm kiếm độ tương đồng hiệu quả trong các tập dữ liệu lớn.
-   `MongoDB`: MongoDB Atlas Vector Search cho phép lưu trữ, lập chỉ mục và truy vấn **embeddings** vector hiệu quả cùng với dữ liệu hoạt động của bạn, tạo điều kiện triển khai liền mạch các ứng dụng dựa trên AI.
-   `pgvector (PostgreSQL)`: Một tiện ích mở rộng cho PostgreSQL bổ sung khả năng tìm kiếm độ tương đồng vector, cho phép lưu trữ và truy vấn dữ liệu vector hiệu quả trong cơ sở dữ liệu quan hệ.
-   `Neo4j`: Một cơ sở dữ liệu đồ thị lưu trữ các node và mối quan hệ, với hỗ trợ gốc cho tìm kiếm vector, tạo điều kiện cho các truy vấn phức tạp liên quan đến cả dữ liệu đồ thị và vector.
-   `Weaviate`: Một cơ sở dữ liệu vector mã nguồn mở cho phép lưu trữ các đối tượng dữ liệu và **embeddings** vector, hỗ trợ nhiều loại dữ liệu khác nhau và cung cấp khả năng tìm kiếm ngữ nghĩa.
-   `Milvus`: Một cơ sở dữ liệu lưu trữ, lập chỉ mục và quản lý các **embeddings** vector khổng lồ được tạo bởi các mô hình học máy, được thiết kế để tìm kiếm độ tương đồng vector hiệu suất cao.

Các **vector stores** này là không thể thiếu trong việc xây dựng các ứng dụng yêu cầu tìm kiếm độ tương đồng hiệu quả và quản lý dữ liệu đa chiều.


| Vector Store             | Delete by ID | Filtering | Search by Vector | Search with Score | Async | Passes Standard Tests | Multi Tenancy | IDs in Add Documents |
|--------------------------|:------------:|:---------:|:----------------:|:-----------------:|:-----:|:---------------------:|:-------------:|:--------------------:|
| Chroma                   |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
| FAISS                    |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
| Pinecone                 |      ✅      |     ✅    |        ✅        |         ❌        |  ✅   |          ❌           |       ❌       |          ❌          |
| Qdrant                   |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
| Elasticsearch            |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
| MongoDB                  |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
| pgvector (PostgreSQL)    |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
| Neo4j                    |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ❌       |          ✅          |
| Weaviate                 |      ✅      |     ✅    |        ✅        |         ✅        |  ✅   |          ❌           |       ✅       |          ❌          |
| Milvus                   |      ✅      |     ✅    |        ❌        |         ✅        |  ✅   |          ❌           |       ❌       |          ❌          |
